
---

### Exercise 3.52. 

Write code to implement the 1D derivative free optimization algorithm and use it to solve Exercise 3.48. Compare your answer to the analytic solution.

Note: For 3.48 we need to find $0<x<10$ that minimizes $V(x) = x (20-2x)^2$.

In [12]:
def deriv_free_min(f, x0, dx):
    """
    Find the approximate minimum of a function using derivative-free optimization.
    This function performs a simple optimization by evaluating the function
    at points to the left and right of the current position and moving in the
    direction that decreases the function value. It stops when no further
    improvement is found.
    Parameters:
    f (callable): The function to minimize. It should take a single argument (x) and return a scalar value.
    x0 (float): The initial guess for the location of the minimum.
    dx (float): The step size used to explore the function to the left and right of the current position.
    Returns:
    tuple: A tuple (current_x, current_f) where:
        - current_x (float): The x-coordinate of the approximate minimum.
        - current_f (float): The function value at the approximate minimum.

    Notes:
    - This method does not use derivatives, so it is suitable for functions
        that are not differentiable or when derivatives are difficult to compute.
    - The choice of `dx` affects the accuracy and speed of convergence.
    - This method assumes the function has a single valley in the region of interest.
    """
    current_x, current_f = x0, f(x0) # initial position and function value
    fevals = 1
    
    while True:
        # Calculate the new positions and function values
        left_x, right_x = current_x - dx, current_x + dx
        left_f, right_f = f(left_x), f(right_x)
        fevals += 2
        
        if left_f < min(current_f, right_f): # f is smaller to the left so "slide" left
            current_x, current_f = left_x, left_f
        elif right_f < min(current_f, left_f): # f is smaller to the right so "slide" right
            current_x, current_f = right_x, right_f
        else: # neither left nor right is better, so we're done
            break

    print(f"Approximate min at x = {current_x:.4f} with f = {current_f:.4f} after {fevals} evaluations.")
    
    return current_x, current_f # return current x and f for the approximate minimum

In [14]:
# Note we're flipping the sign of the function to find the maximum
V = lambda x: -x*(20-2*x)**2
x0 = 2
dx = 0.1
x, Vx = deriv_free_min(V, x0, dx)

Approximate min at x = 3.3000 with f = -592.5480 after 29 evaluations.


In [16]:
def deriv_free_min_v2(f, x0, dx):
    """
    Find the approximate minimum of a function using derivative-free optimization.
    This function performs a simple optimization by evaluating the function
    at points to the left and right of the current position and moving in the
    direction that decreases the function value. It stops when no further
    improvement is found.
    Parameters:
    f (callable): The function to minimize. It should take a single argument (x) and return a scalar value.
    x0 (float): The initial guess for the location of the minimum.
    dx (float): The step size used to explore the function to the left and right of the current position.
    Returns:
    tuple: A tuple (current_x, current_f) where:
        - current_x (float): The x-coordinate of the approximate minimum.
        - current_f (float): The function value at the approximate minimum.

    Notes:
    - This method does not use derivatives, so it is suitable for functions
        that are not differentiable or when derivatives are difficult to compute.
    - The choice of `dx` affects the accuracy and speed of convergence.
    - This method assumes the function has a single valley in the region of interest.
    """

    left_x, current_x, right_x = x0 - dx, x0, x0 + dx # initial positions
    left_f, current_f, right_f = f(left_x), f(current_x), f(right_x) # initial function values 
    fevals = 3
    
    while True:
        if left_f < min(current_f, right_f): # f is smaller to the left so "slide" left
            current_x, current_f, right_x, right_f = left_x, left_f, current_x, current_f
            left_x = current_x - dx
            left_f = f(left_x)
            fevals += 1
        elif right_f < min(current_f, left_f): # f is smaller to the right so "slide" right
            current_x, current_f, left_x, left_f = right_x, right_f, current_x, current_f
            right_x = current_x + dx
            right_f = f(right_x)
            fevals += 1
        else: # neither left nor right is better, so we're done
            break
    
    print(f"Approximate min at x = {current_x:.4f} with f = {current_f:.4f} after {fevals} evaluations.")
    
    return current_x, current_f # return current x and f for the approximate minimum

In [17]:
# Note we're flipping the sign of the function to find the maximum
V = lambda x: -x*(20-2*x)**2
x0 = 2
dx = 0.1
x, Vx = deriv_free_min_v2(V, x0, dx)

Approximate min at x = 3.3000 with f = -592.5480 after 16 evaluations.


In [18]:
def deriv_free_min_adapt_dx(f, x0, dx, dx_min=1e-6):
    """
    Find the approximate minimum of a function using derivative-free optimization with adaptive step size.
    
    This method evaluates the function at points to the left and right of the current position 
    and moves in the direction that decreases the function value. If no improvement is found, 
    the step size is halved until it reaches a minimum threshold (dx_min).
    
    Parameters:
    f (callable): The function to minimize. It should take a single argument (x) and return a scalar value.
    x0 (float): The initial guess for the location of the minimum.
    dx (float): The initial step size used to explore the function to the left and right of the current position.
    dx_min (float): The minimum allowable step size. Default is 1e-6.
    
    Returns:
    tuple: A tuple (current_x, current_f) where:
        - current_x (float): The x-coordinate of the approximate minimum.
        - current_f (float): The function value at the approximate minimum.
    
    Notes:
    - This method does not use derivatives, so it is suitable for functions that are not differentiable 
      or when derivatives are difficult to compute.
    - The choice of `dx` affects the accuracy and speed of convergence.
    - This method assumes the function has a single valley in the region of interest.
    """

    left_x, current_x, right_x = x0 - dx, x0, x0 + dx # initial positions
    left_f, current_f, right_f = f(left_x), f(current_x), f(right_x) # initial function values 
    fevals = 3
    
    while True:
        if left_f < min(current_f, right_f): # f is smaller to the left so "slide" left
            current_x, current_f, right_x, right_f = left_x, left_f, current_x, current_f
            left_x = current_x - dx
            left_f = f(left_x)
            fevals += 1
        elif right_f < min(current_f, left_f): # f is smaller to the right so "slide" right
            current_x, current_f, left_x, left_f = right_x, right_f, current_x, current_f
            right_x = current_x + dx
            right_f = f(right_x)
            fevals += 1
        else:
            if dx < dx_min:  # Stop if the step size is below the minimum threshold
                break
            dx = dx / 2  # reduce the step size by half
            left_x, right_x = current_x - dx, current_x + dx # update the new positions
            left_f, right_f = f(left_x), f(right_x) # update the new function values
            fevals += 2
    
    print(f"Approximate min at x = {current_x:.4f} with f = {current_f:.4f} after {fevals} evaluations.")

    return current_x, current_f # return current x and f for the approximate minimum

In [20]:
# Note we're flipping the sign of the function to find the maximum
V = lambda x: -x*(20-2*x)**2
x0 = 2
dx = 0.1
x, Vx = deriv_free_min_adapt_dx(V, x0, dx, dx_min= 1e-4)

Approximate min at x = 3.3333 with f = -592.5926 after 46 evaluations.
